**ライブラリの読み込み**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**前処理**

In [ ]:
#データの読み込み（前後3ヶ月）
df06 = pd.read_csv("/content/drive/MyDrive/POSdata/074_緑茶飲料_202206.csv", encoding='shift-jis')
df07 = pd.read_csv("/content/drive/MyDrive/POSdata/074_緑茶飲料_202207.csv", encoding='shift-jis')
df08 = pd.read_csv("/content/drive/MyDrive/POSdata/074_緑茶飲料_202208.csv", encoding='shift-jis')

In [ ]:
#追加データの読み込み
#変数
#・MM：POSデータと同様
#・DD：POSデータと同様
#・平均気温
#・平均湿度
df_add = pd.read_csv("add.csv", encoding='shift-jis')

In [ ]:
#データの結合
df = pd.concat([df06,df07,df08],axis=0)

In [ ]:
#店舗IDをつける
df['店舗ID'] = 0

In [ ]:
before_DD = 0
shopID = 0

for index, row in df.iterrows():
  if before_DD > row['DD']:
    shopID = shopID + 1
  df.at[index, '店舗ID'] = shopID
  before_DD = row['DD']

In [ ]:
df.columns

Index(['店舗立地', 'レシート番号', 'YYYY', 'MM', 'DD', '曜日フラグ', '休日フラグ', 'hh', 'mm',
       'ss', '購入者性別フラグ', '購入者年齢フラグ', '購入商品名', 'JANコード', '単価', '個数', '値段',
       'メーカー名', '分類コード', '分類名', '店舗ID'],
      dtype='object')

In [ ]:
df2 = pd.merge(df,df_add)

In [ ]:
df2['店舗ID'] = df2['店舗ID'] - 31

In [ ]:
#曜日の数字をカテゴリに変える
week = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
for i in range(1, 8):
  df2["曜日フラグ"] = df2["曜日フラグ"].replace(i, week[i-1])

In [ ]:
df2

,店舗立地,レシート番号,YYYY,MM,DD,曜日フラグ,休日フラグ,hh,mm,ss,...,JANコード,単価,個数,値段,メーカー名,分類コード,分類名,店舗ID,平均気温,平均湿度
0,オフィス街,12945,2022,6,1,Wed,NaN,6,14,49,...,49151947,144,1,144,サントリーホールディングス,76004.0,ペットボトル入りウーロン茶飲料,1,20.1,60
1,オフィス街,12945,2022,6,1,Wed,NaN,6,14,49,...,4901085191927,119,1,119,伊藤園,74004.0,ペットボトル入り緑茶飲料,1,20.1,60
2,オフィス街,12945,2022,6,1,Wed,NaN,6,14,49,...,9999999151485,139,1,139,ＰＢ,14014.0,おにぎり・おむすび,1,20.1,60
3,オフィス街,12945,2022,6,1,Wed,NaN,6,14,49,...,9999999176983,3,1,3,ＰＢ,900000.0,新規アイテム,1,20.1,60
4,オフィス街,12945,2022,6,1,Wed,NaN,6,14,49,...,9999999197438,146,1,146,ＰＢ,14014.0,おにぎり・おむすび,1,20.1,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293558,住宅街,324714,2022,8,31,Wed,NaN,23,20,46,...,4901777380004,138,1,138,サントリーホールディングス,79014.0,ペットボトル入り果汁入り清涼飲料,16,29.0,82
293559,住宅街,324714,2022,8,31,Wed,NaN,23,20,46,...,4902102107648,119,1,119,日本コカ・コーラ,74004.0,ペットボトル入り緑茶飲料,16,29.0,82
293560,住宅街,324714,2022,8,31,Wed,NaN,23,20,46,...,4902663014591,147,1,147,ひかり味噌,144011.0,即席カップ入りみそ汁・和風汁,16,29.0,82
293561,住宅街,324714,2022,8,31,Wed,NaN,23,20,46,...,4902720148382,150,1,150,森永乳業,261999.0,その他レギュラーアイス,16,29.0,82


パターン１

In [ ]:
#データの準備（店舗ID）
df_num = df2.groupby(['店舗ID','DD'])['レシート番号'].nunique()
df_sales = df2.groupby(['店舗ID','DD'])['値段'].sum()
df_gender = df2.groupby(['店舗ID','DD'])['購入者性別フラグ'].first()
df_place = df2.groupby(['店舗ID','DD'])['店舗立地'].first()
df_week = df2.groupby(['店舗ID','DD'])['曜日フラグ'].first()
df_holiday = df2.groupby(['店舗ID','DD'])['休日フラグ'].first()
df_temperature = df2.groupby(['店舗ID','DD'])['平均気温'].first()
df_humidity = df2.groupby(['店舗ID','DD'])['平均湿度'].first()

In [ ]:
df1_3 = pd.concat([df_sales, df_num,df_week, df_gender, df_place, df_holiday,df_temperature,df_humidity],axis=1,ignore_index=False)

In [ ]:
#カテゴリカルデータの数値化
df1_4 = pd.get_dummies(df1_3,drop_first=True)

In [ ]:
df1_5 = df1_4.fillna(0)

In [ ]:
df1_6 = df1_5.drop(['休日フラグ','平均気温',],axis=1)

パターン２

In [ ]:
#データの準備（性別）
df_num2 = df2.groupby(['購入者性別フラグ','DD'])['レシート番号'].nunique()
df_sales2 = df2.groupby(['購入者性別フラグ','DD'])['値段'].sum()
df_gender2 = df2.groupby(['購入者性別フラグ','DD'])['店舗立地'].first()
df_place2 = df2.groupby(['購入者性別フラグ','DD'])['店舗ID'].first()
df_week2 = df2.groupby(['購入者性別フラグ','DD'])['曜日フラグ'].first()
df_holiday2 = df2.groupby(['購入者性別フラグ','DD'])['休日フラグ'].first()
df_temperature2 = df2.groupby(['購入者性別フラグ','DD'])['平均気温'].first()
df_humidity2 = df2.groupby(['購入者性別フラグ','DD'])['平均湿度'].first()

In [ ]:
df2_3 = pd.concat([df_sales2, df_num2,df_week2, df_gender2, df_place2, df_holiday2,df_temperature2,df_humidity2],axis=1,ignore_index=False)

In [ ]:
#カテゴリカルデータの数値化
df2_4 = pd.get_dummies(df2_3,drop_first=True)

In [ ]:
df2_5 = df2_4.fillna(0)

In [ ]:
df2_6 = df2_5.drop(['休日フラグ','平均気温',],axis=1)

In [ ]:
df2_6

値段  レシート番号  店舗ID  平均湿度  曜日フラグ_Mon  曜日フラグ_Sat  曜日フラグ_Sun  \
購入者性別フラグ DD                                                                 
1        1   1519497    2421     1    60          0          0          0   
         2   1434828    2102     1    57          0          0          0   
         3   1237834    1852     1    64          0          0          0   
         4   1403473    1995     1    53          0          1          0   
         5   1231851    1802     1    73          0          0          1   
...              ...     ...   ...   ...        ...        ...        ...   
2        27   545023     748     2    79          1          0          0   
         28   291047     403     1    74          0          0          0   
         29   344546     559     1    71          0          0          0   
         30   581012     785     1    66          0          0          0   
         31   287244     391     2    70          0          0          1   

             曜日フラグ_Thu  曜日フラグ_Tue  曜日フラグ_Wed  
購入者性別フラグ DD                                   
1        1           0          0          1  
         2           1          0          0  
         3           0          0          0  
         4           0          0          0  
         5           0          0          0  
...                ...        ...        ...  
2        27          0          0          0  
         28          1          0          0  
         29          0          0          0  
         30          1          0          0  
         31          0          0          0  

[62 rows x 10 columns]

**回帰モデル**

パターン１

In [ ]:
value = pd.DataFrame(index=range(1,1001),columns=['係数','切片','学習モデルの決定係数','テストデータの決定係数','MAE','MSE'])

In [ ]:
#学習モデルの決定係数とテストデータの決定係数の差の最小値
min = 1 #決定係数の最大値
tmp_i = 0 #決定係数の差が最小の時のrandom_stateの値
tmp = 0

for i in range(1,1001):
  #学習７割、テスト３割
  df_train, df_test = train_test_split(df1_6, test_size=0.3,random_state=i)

  #学習データとテストデータの目的変数と説明変数の作成
  #目的変数y：値段
  #説明変数X：その他
  X_train = df_train.drop('値段', axis=1)
  y_train = df_train['値段']
  X_test = df_test.drop('値段', axis=1)
  y_test = df_test['値段']

  #モデルのインスタンスの作成
  model = LinearRegression()

  #回帰分析の実行
  model.fit(X_train, y_train)

  #係数
  value.at[i,'係数'] = model.coef_

  # 切片
  value.at[i,'切片'] = model.intercept_

  #学習モデルの決定係数
  value.at[i,'学習モデルの決定係数'] = model.score(X_train, y_train)
  r_train = model.score(X_train, y_train)

  #テストデータの決定係数
  value.at[i,'テストデータの決定係数'] = model.score(X_test, y_test)
  r_test = model.score(X_test, y_test)

  #テストデータの予測
  y_pred = model.predict(X_test)

  #平均絶対誤差 (MAE)
  value.at[i,'MAE'] = mean_absolute_error(y_test, y_pred)

  #平均二乗誤差 (MSE)
  value.at[i,'MSE'] = mean_squared_error(y_test, y_pred)

  #決定係数の差
  if r_train >= r_test:
    tmp = r_train - r_test
  else:
    tmp = r_test - r_train

  if min > tmp:
    min = tmp
    tmp_i = i

print('決定係数の差の最小値：',min)
print('その時のrandom_stateの値：',tmp_i)
print('その時の学習モデルの決定係数',value.at[tmp_i,'学習モデルの決定係数'])
print('その時のテストデータの決定係数',value.at[tmp_i,'テストデータの決定係数'])
print('その時の平均絶対誤差 (MAE)：',value.at[tmp_i,'MAE'])
print('その時の平均二乗誤差 (MSE)：',value.at[tmp_i,'MSE'])

決定係数の差の最小値： 0.00015344084201063257
その時のrandom_stateの値： 511
その時の学習モデルの決定係数 0.8841312805773394
その時のテストデータの決定係数 0.88428472141935
その時の平均絶対誤差 (MAE)： 16611.58458778577
その時の平均二乗誤差 (MSE)： 659870132.6339313


In [ ]:
print('[指標の平均]')
print('学習モデルの決定係数：',value.学習モデルの決定係数.mean())
print('テストデータの決定係数：',value.テストデータの決定係数.mean())
print('平均絶対誤差 (MAE)：',value.MAE.mean())
print('平均二乗誤差 (MSE)：',value.MSE.mean())

[指標の平均]
学習モデルの決定係数： 0.8878154153883299
テストデータの決定係数： 0.8789722638024904
平均絶対誤差 (MAE)： 16569.86234991848
平均二乗誤差 (MSE)： 594963072.9524009


パターン２

In [ ]:
value2 = pd.DataFrame(index=range(1,1001),columns=['係数','切片','学習モデルの決定係数','テストデータの決定係数','MAE','MSE'])

In [ ]:
#学習モデルの決定係数とテストデータの決定係数の差の最小値
min2 = 1 #決定係数の最大値
tmp_i2 = 0 #決定係数の差が最小の時のrandom_stateの値
tmp2 = 0

for i2 in range(1,1001):
  #学習７割、テスト３割
  df_train2, df_test2 = train_test_split(df2_6, test_size=0.3,random_state=i2)

  #学習データとテストデータの目的変数と説明変数の作成
  #目的変数y：値段
  #説明変数X：その他
  X_train2 = df_train2.drop('値段', axis=1)
  y_train2 = df_train2['値段']
  X_test2 = df_test2.drop('値段', axis=1)
  y_test2 = df_test2['値段']

  #モデルのインスタンスの作成
  model2 = LinearRegression()

  #回帰分析の実行
  model2.fit(X_train2, y_train2)

  #係数
  value2.at[i2,'係数'] = model2.coef_

  # 切片
  value2.at[i2,'切片'] = model2.intercept_

  #学習モデルの決定係数
  value2.at[i2,'学習モデルの決定係数'] = model2.score(X_train2, y_train2)
  r_train2 = model2.score(X_train2, y_train2)

  #テストデータの決定係数
  value2.at[i2,'テストデータの決定係数'] = model2.score(X_test2, y_test2)
  r_test2 = model2.score(X_test2, y_test2)

  #テストデータの予測
  y_pred2 = model2.predict(X_test2)

  #平均絶対誤差 (MAE)
  value2.at[i2,'MAE'] = mean_absolute_error(y_test2, y_pred2)

  #平均二乗誤差 (MSE)
  value2.at[i2,'MSE'] = mean_squared_error(y_test2, y_pred2)

  #決定係数の差
  if r_train2 >= r_test2:
    tmp2 = r_train2 - r_test2
  else:
    tmp2 = r_test2 - r_train2

  if min2 > tmp2:
    min2 = tmp2
    tmp_i2 = i2

print('決定係数の差の最小値：',min2)
print('その時のrandom_stateの値：',tmp_i2)
print('その時の学習モデルの決定係数',value2.at[tmp_i2,'学習モデルの決定係数'])
print('その時のテストデータの決定係数',value2.at[tmp_i2,'テストデータの決定係数'])
print('その時の平均絶対誤差 (MAE)：',value2.at[tmp_i2,'MAE'])
print('その時の平均二乗誤差 (MSE)：',value2.at[tmp_i2,'MSE'])

決定係数の差の最小値： 1.249140102099755e-05
その時のrandom_stateの値： 207
その時の学習モデルの決定係数 0.9864320263197951
その時のテストデータの決定係数 0.9864445177208161
その時の平均絶対誤差 (MAE)： 33341.19283668869
その時の平均二乗誤差 (MSE)： 2322003744.6524587


In [ ]:
print('[指標の平均]')
print('学習モデルの決定係数：',value2.学習モデルの決定係数.mean())
print('テストデータの決定係数：',value2.テストデータの決定係数.mean())
print('平均絶対誤差 (MAE)：',value2.MAE.mean())
print('平均二乗誤差 (MSE)：',value2.MSE.mean())

[指標の平均]
学習モデルの決定係数： 0.988952361111767
テストデータの決定係数： 0.9792536504755759
平均絶対誤差 (MAE)： 40693.78897346811
平均二乗誤差 (MSE)： 3336667438.1051097
